In [2]:
import numpy as np

In [2]:
data = np.load('/home/peijia/TransUNet/case0005_slice000.npz')
image, label = data['image'], data['label']

In [3]:
image.shape

(512, 512)

In [4]:
label.shape

(512, 512)

In [13]:
image.dtype

dtype('float32')

In [14]:
label.dtype

dtype('float32')

In [16]:
unique_values, counts = np.unique(label, return_counts=True)

In [18]:
counts

array([262144])

In [7]:
a = open('/home/peijia/medical_dataset/train_img_1.txt').readlines()

In [20]:
name = a[0]

In [21]:
name.strip('\n')

'0865_image.nii.gz'

In [10]:
b = open('/home/peijia/medical_dataset/train_mask_1.txt').readlines()

In [11]:
b[0]

'0865_mask_1.nii.gz\n'

In [12]:
[1]+[2]

[1, 2]

In [23]:
import SimpleITK as sitk

In [25]:
img = sitk.ReadImage(str('/home/peijia/medical_dataset/image/0852_image.nii.gz'))
a = sitk.GetArrayFromImage(img)


In [26]:
a.shape

(17, 512, 512)

In [27]:
a[1].shape

(512, 512)

In [3]:
arr = np.ones([19, 512, 512])

In [4]:
import torch
arr = torch.tensor(arr)

/home/peijia/miniconda3/envs/medical/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
h, w = arr.size()[1:]

In [9]:
h

512